In [1]:
from sentence_transformers import SentenceTransformer

ModuleNotFoundError: No module named 'sentence_transformers'

In [2]:
pip install sentence_transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [3]:
pip install singlestoredb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.7 MB/s eta 0:00:00


In [6]:
from transformers import CLIPTokenizer, CLIPTextModel
import torch
import singlestoredb as s2

In [ ]:
import os

from PIL import Image
import numpy as np
from sentence_transformers import SentenceTransformer

PATH = os.getcwd()
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu").type


def convert_repo_to_embedding(path:str) -> dict:
  img_model = SentenceTransformer('clip-ViT-B-32', device=DEVICE)
  image_paths = os.listdir(path)

  embeddings = img_model.encode([Image.open(os.path.join(PATH, image_path)) for image_path in image_paths], show_progress_bar=True, device=DEVICE)

  del img_model
  return {image_path: embedding for image_path, embedding in zip(image_paths, embeddings)}

In [ ]:
DEVICE

'cpu'

In [7]:
pip install torch

In [12]:
import numpy as np
import singlestoredb as s2
from PIL import Image
from sentence_transformers import SentenceTransformer
import os
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def generate_embeddings(image_path):
    """Generate embeddings for a given image."""
    img_model = SentenceTransformer('clip-ViT-B-32', device=DEVICE)
    image = Image.open(image_path)
    return img_model.encode([image], show_progress_bar=False, device=DEVICE)[0]

def store_embeddings_db(images_folder: str) -> None:
    conn = s2.connect('admin:g8xw7aEVyFxOozJYPUdhh5PRFoAjfybK@svc-eaabb9b0-ceaf-4832-a39a-2fdad892e1ad-dml.aws-oregon-3.svc.singlestore.com:3306/sarasvdb')

    with conn:
        with conn.cursor() as cur:
            cur.execute('''CREATE TABLE IF NOT EXISTS image_embeddings (
                            id INT AUTO_INCREMENT PRIMARY KEY,
                            image_name VARCHAR(255),
                            image_path VARCHAR(255),
                            embedding BLOB
                           )''')

            image_names = os.listdir(images_folder)
            new_images = []
            embeddings_to_return = []

            for image_name in image_names:
                image_path = os.path.join(images_folder, image_name)

                cur.execute("SELECT embedding FROM image_embeddings WHERE image_path = %s",
                            (image_path,))
                result = cur.fetchone()

                if result:
                    embeddings_to_return.append((image_name, result[0]))
                else:
                    embeddings = generate_embeddings(image_path)
                    embeddings_to_return.append((image_name, embeddings))
                    new_images.append((image_name, image_path, embeddings.tobytes()))

            if new_images:
                cur.executemany(
                    """INSERT INTO image_embeddings (image_name, image_path, embedding)
                    VALUES (%s, %s, %s)
                    ON DUPLICATE KEY UPDATE embedding=VALUES(embedding)""",
                    new_images
                )
                conn.commit()

    return embeddings_to_return

images_folder = '/content/images_folder'
result = store_embeddings_db(images_folder)
print(result)


[('Screenshot 2024-03-14 152011.png', array([ 5.81437826e-01,  1.96903855e-01, -2.62789130e-01,  1.91699177e-01,
       -3.63245189e-01, -5.84146321e-01, -1.25363618e-01,  9.94818658e-02,
        6.35490477e-01, -1.08008832e-03, -2.52061725e-01,  1.63439959e-01,
       -6.00544035e-01,  1.72432065e-01, -2.03249574e-01, -3.84760231e-01,
        2.70684421e-01, -1.25908762e-01, -4.53342170e-01,  2.76789784e-01,
       -1.65565893e-01,  4.06449884e-02, -2.84223557e-02,  2.40758747e-01,
        8.47901851e-02,  6.48243308e-01, -1.86373129e-01,  1.44442454e-01,
       -2.19886333e-01, -4.36589986e-01,  4.23684716e-01,  2.51125157e-01,
        3.70173901e-01,  4.72041428e-01,  9.14023399e-01, -3.66629392e-01,
        4.32471037e-01,  7.37214833e-02,  5.52011877e-02, -1.89988434e+00,
       -4.46793288e-02,  2.46258661e-01, -7.40177184e-02, -3.18612248e-01,
        1.47785857e-01, -8.35658193e-01,  7.05789998e-02, -1.96732879e-02,
        2.18274951e-01, -1.32890612e-01,  7.85744861e-02, -5.6

In [19]:
def get_highest_similarity_image(text: str, top_n: int) -> list:
    """Retrieve the top N images with the highest similarity to the given text."""
    try:
        with s2.connect('admin:g8xw7aEVyFxOozJYPUdhh5PRFoAjfybK@svc-eaabb9b0-ceaf-4832-a39a-2fdad892e1ad-dml.aws-oregon-3.svc.singlestore.com:3306/sarasvdb') as conn:
            with conn.cursor() as cur:
                cur.execute('SELECT image_path, embedding FROM image_embeddings')
                image_embeddings = cur.fetchall()

    except Exception as e:
        print(f"Error connecting to database or fetching data: {e}")
        return []

    try:
        text_embed = get_text_embedding(text)
        text_embed = normalize(text_embed)

        similarities = []
        for image_path, embedding in image_embeddings:
            image_embed = np.frombuffer(embedding, dtype=np.float32)
            image_embed = normalize(image_embed)
            similarity_score = get_similarity_score(text_embed, image_embed)
            similarities.append((image_path, similarity_score))

        similarities.sort(key=lambda x: x[1], reverse=False)
        return similarities[:top_n]

    except Exception as e:
        print(f"Error processing embeddings or computing similarities: {e}")
        return []

if __name__ == "__main__":
    # Take input from the user
    user_text = input("Enter the text to find similar images: ")
    top_n = int(input("Enter the number of top similar images to retrieve: "))

    # Call the function to get top N similar images
    similar_images = get_highest_similarity_image(user_text, top_n)

    # Display the results
    if similar_images:
        print(f"\nTop {top_n} images most similar to '{user_text}':")
        for idx, (image_path, score) in enumerate(similar_images, 1):
            print(f"{idx}. {image_path} - Similarity Score: {score}")
    else:
        print("No similar images found or an error occurred.")


Enter the text to find similar images: An image with color full 3 dimensional graph
Enter the number of top similar images to retrieve: 4

Top 4 images most similar to 'An image with color full 3 dimensional graph':
1. /content/images_folder/Screenshot 2024-03-14 152011.png - Similarity Score: [-26.474854]
2. /content/images_folder/Screenshot 2024-03-13 131410.png - Similarity Score: [-24.117271]
3. /content/images_folder/Screenshot 2024-03-14 151752.png - Similarity Score: [-23.008867]
4. /content/images_folder/Screenshot 2024-03-13 131425.png - Similarity Score: [-19.71877]


In [14]:
def normalize(vector: np.array) -> np.array:
    """Normalize the vector."""
    mean = vector.mean()
    std = vector.std()
    if std == 0:
        return vector - mean
    return (vector - mean) / std

In [15]:
def get_similarity_score(vector1: np.array, vector2: np.array) -> float:
    """Calculate cosine similarity between two vectors."""
    return np.dot(normalize(vector1), normalize(vector2))

In [17]:
def get_text_embedding(text):

  model_name = 'openai/clip-vit-base-patch32'
  tokenizer = CLIPTokenizer.from_pretrained(model_name)
  model = CLIPTextModel.from_pretrained(model_name)

  """Generate an embedding for the given text."""
  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
  with torch.no_grad():
      outputs = model(**inputs)
  # Extract the last hidden state
  embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy()

  del tokenizer
  del model

  return embedding